# **The Tokenizers**
**Train** a brand **new tokenizer on a corpus of texts**, so it can then be **used to pretrain a language model**. This will all be done with the help of the 🤗 `Tokenizers` library, which provides the **“fast”** tokenizers in the 🤗 Transformers library. We’ll take a close look at the features that this library provides, and explore how the fast tokenizers differ from the **“slow”** versions.

**Topics:**
- How to **train a new tokenizer** similar to the one used by a given checkpoint on a new corpus of texts
- The special features of **fast tokenizers**
- The differences between the **three main subword tokenization algorithms** used in NLP today
- How to **build a tokenizer from scratch** with the 🤗 Tokenizers library and train it on some data.

## **Training a New Tokenizer from an old one**
**If a language model is not available in the language you are interested in**, *or* **if your corpus is very different from the one your language model was trained on**, you will most likely want to **retrain the model from scratch using a tokenizer adapted to your data**. That will require training a new tokenizer on your dataset.


Most Transformers models use a *`subword tokenization algorithm`*. To identify which subwords are of interest and occur most frequently in the corpus at hand, the **`tokenizer` needs to take a hard look at all the texts in the corpus** — a process we call *training*. The exact rules that govern this training depend on **the type of tokenizer used** (*go over 3 main algorithm*).

⚠️ **Training a tokenizer** is not the same as training a model! Model training uses *stochastic gradient descent* to make the loss a little bit smaller for each batch. It’s randomized by nature (meaning you have to set some seeds to get the same results when doing the same training twice).

⚠️ **Training a tokenizer** is a statistical process that tries to **identify which subwords are the best to pick for a given corpus**, *and* **the exact rules used to pick them depend on the tokenization algorithm**. It’s deterministic, meaning you *always get the same results when training with the same algorithm on the same corpus*.

### **Assembling a corpus**
There’s a very simple API in 🤗 Transformers that you can use to **train a new tokenizer with the same characteristics as an existing one**: `AutoTokenizer.train_new_from_iterator()`.

To see this in action, let’s say we want to **train `GPT-2` from scratch, but in a language other than *English*.** Our *first task* will be to **gather lots of data in that language in a training corpus**. To provide examples everyone will be able to understand, we won’t use a language like Russian or Chinese here, but rather a specialized English language: ***Python code.***

The 🤗 Datasets library can help us assemble a corpus of Python source code. We’ll use the usual `load_dataset()` function to download and cache the `CodeSearchNet` dataset. This dataset was created for the CodeSearchNet challenge and **contains millions of functions from open source libraries on GitHub in several programming languages**. Here, we will load the **Python** part of this dataset:

In [ ]:
from datasets import load_dataset

# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("code_search_net", "python")

In [ ]:
raw_datasets["train"]

## Output
# Dataset({
#     features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language',
#       'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name',
#       'func_code_url'
#     ],
#     num_rows: 412178
# })

In [ ]:
raw_datasets["train"][0]

Here. we’ll just use the `whole_func_string` column to train our tokenizer.

In [ ]:
print(raw_datasets["train"][123456]["whole_func_string"])

## Output
# def handle_simple_responses(
#       self, timeout_ms=None, info_cb=DEFAULT_MESSAGE_CALLBACK):
#     """Accepts normal responses from the device.

#     Args:
#       timeout_ms: Timeout in milliseconds to wait for each response.
#       info_cb: Optional callback for text sent from the bootloader.

#     Returns:
#       OKAY packet's message.
#     """
#     return self._accept_responses('OKAY', info_cb, timeout_ms=timeout_ms)

The first thing we need to do is **transform the dataset into an *iterator* of lists of texts** — for instance, a list of list of texts.

Using lists of texts will enable our tokenizer to go faster (training on batches of texts instead of processing individual texts one by one), and it should be an iterator if we want to avoid having everything in memory at once.

If your corpus is huge, you will want to take advantage of the fact that *🤗 `Datasets` does not load everything into RAM but stores the elements of the dataset on disk*.

Create **list of lists of 1000 texts each** by using a *Python generator*, we can avoid Python loading anything into memory until it’s actually necessary.

In [ ]:
# Don't uncomment the following line unless your dataset is small!
# training_corpus = [raw_datasets["train"][i: i + 1000]["whole_func_string"] for i in range(0, len(raw_datasets["train"]), 1000)]

In [ ]:
## Using Python Generator
training_corpus = (
    raw_datasets["train"][i : i + 1000]["whole_func_string"]
    for i in range(0, len(raw_datasets["train"]), 1000)
)
# This code doesn’t fetch any elements of the dataset;
# it just creates an object you can use in a Python for loop

The texts will only be loaded when you need them (that is, when you’re at the step of the `for` loop that requires them), and only 1,000 texts at a time will be loaded.

The problem with a *generator object* is that it can only be *used once*. So, instead of this giving us the list of the first 10 digits twice, we get them **once and then an empty list**.

In [ ]:
## Define the function to returns a generator instead
def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["whole_func_string"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )


training_corpus = get_training_corpus()

In [ ]:
# or define a generator inside a for loop by using the yield
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]

# which will produce the exact same generator as before,
# but allows you to use more complex logic than you can in a list comprehension.

### **Training a new Tokenizer**
Now that we have our **corpus** in the form of an *iterator of batches of texts*, we are ready to train a new tokenizer.

Load the tokenizer that want to pair with our model (etc., GPT-2)

In [ ]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

Even though we are going to train a new tokenizer, it’s a good idea to do this to avoid starting entirely from scratch.

This way, we won’t have to specify anything about the tokenization algorithm or the special tokens we want to use; **our new tokenizer will be exactly the same as GPT-2**, *and* **the only thing that will change is the vocabulary**, which will be **determined by the training on our corpus**.

In [ ]:
## Example
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

# ['def', 'Ġadd', '_', 'n', 'umbers', '(', 'a', ',', 'Ġb', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo',
#  'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`', '."', '""', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']

This tokenizer has a few special symbols, like `Ġ` and `Ċ`, which denote *spaces* and *newlines*, respectively.

As we can see, this is not too efficient: the tokenizer returns individual tokens for each space, when it could group together indentation levels (since having sets of four or eight spaces is going to be very common in code). It also split the function name a bit weirdly, not being used to seeing words with the `_` character.

Let’s train a new tokenizer and see if it solves those issues. For this, we’ll use the method `train_new_from_iterator()`

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [ ]:
tokens = tokenizer.tokenize(example)
tokens

# ['def', 'Ġadd', '_', 'numbers', '(', 'a', ',', 'Ġb', '):', 'ĊĠĠĠ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`',
#  'a', '`', 'Ġand', 'Ġ`', 'b', '`."""', 'ĊĠĠĠ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']

Here we again see the special symbols `Ġ` and `Ċ` that denote *spaces* and *newlines*, but we can also see that our tokenizer learned some tokens that are highly specific to a corpus of Python functions: for example, there is a `ĊĠĠĠ` token that represents an *indentation*, and a `Ġ"""` token that represents the *three quotes that start a docstring*. The tokenizer also correctly split the function name on `_`.

This is quite a compact representation; comparatively, using the plain English tokenizer on the same example will give us a longer sentence:

In [ ]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

In [ ]:
# Another example
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
tokenizer.tokenize(example)

# ['class', 'ĠLinear', 'Layer', '():', 'ĊĠĠĠ', 'Ġdef', 'Ġ__', 'init', '__(', 'self', ',', 'Ġinput', '_', 'size', ',',
#  'Ġoutput', '_', 'size', '):', 'ĊĠĠĠĠĠĠĠ', 'Ġself', '.', 'weight', 'Ġ=', 'Ġtorch', '.', 'randn', '(', 'input', '_',
#  'size', ',', 'Ġoutput', '_', 'size', ')', 'ĊĠĠĠĠĠĠĠ', 'Ġself', '.', 'bias', 'Ġ=', 'Ġtorch', '.', 'zeros', '(',
#  'output', '_', 'size', ')', 'ĊĊĠĠĠ', 'Ġdef', 'Ġ__', 'call', '__(', 'self', ',', 'Ġx', '):', 'ĊĠĠĠĠĠĠĠ',
#  'Ġreturn', 'Ġx', 'Ġ@', 'Ġself', '.', 'weights', 'Ġ+', 'Ġself', '.', 'bias', 'ĊĠĠĠĠ']

In addition to the token corresponding to an indentation, here we can also see a token for a double indentation: `ĊĠĠĠĠĠĠĠ`. The special Python words like `class`, `init`, `call`, `self`, and `return` are each tokenized as one token, and we can see that as well as splitting on `_` and `.` the tokenizer correctly splits even camel-cased names: LinearLayer is tokenized as `["ĠLinear", "Layer"].`

### **Saving the Tokenizer**
To make sure we can use it later, we need to save our new tokenizer. Like for models, this is done with the `save_pretrained()` method

In [ ]:
tokenizer.save_pretrained("code-search-net-tokenizer")

This will create a new folder named `code-search-net-tokenizer`, which will contain all the files the tokenizer needs to be reloaded.

If you want to share this tokenizer with your colleagues and friends, you can upload it to the Hub by logging into your account

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Working with terminal
# huggingface-cli login

**Push tokenizer to the Hub**

In [ ]:
tokenizer.push_to_hub("code-search-net-tokenizer")

This will create a new repository in your namespace with the name `code-search-net-tokenizer`, containing the tokenizer file.

In [ ]:
## Load our new Tokenizer
# Replace "huggingface-course" below with your actual namespace to use your own tokenizer
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")

## **Fast Tokenizers' special powers**
Take a closer look at the capabilities of the tokenizers in 🤗 Transformers. Up to now we have only used them to tokenize inputs or decode IDs back into text, but tokenizers — especially those backed by the 🤗 Tokenizers library — **can do a lot more**.

Exploring how to **reproduce the results** of the `token-classification` (that we called `ner`) and `question-answering` pipelines

**Slow** tokenizers are those *written in Python* inside the 🤗 Transformers library, while the **fast** versions are the ones provided by 🤗 Tokenizers, which are *written in Rust*.

### **Batch Encoding**
The output of a tokenizer isn’t a simple Python dictionary; what we get is actually a special `BatchEncoding` object. It’s a subclass of a dictionary (which is why we were able to index into that result without any problem before), but with additional methods that are mostly used by **fast tokenizers**.

Besides their *parallelization* capabilities, the key functionality of **fast tokenizers** is that they always **keep track of the original span of texts the final tokens come from** — a feature we call *offset mapping*. This in turn unlocks features like mapping each word to the tokens it generated or mapping each character of the original text to the token it’s inside, and vice versa.

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
encoding = tokenizer(example)
print(type(encoding))

## output
# <class 'transformers.tokenization_utils_base.BatchEncoding'>

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

<class 'transformers.tokenization_utils_base.BatchEncoding'>


Since the `AutoTokenizer` class picks a **fast tokenizer** by default, we can use the additional methods this `BatchEncoding` object provides.

We have two ways to check if our tokenizer is a fast or a slow one. We can either check the attribute `is_fast` of the `tokenizer`

In [2]:
# is_fast
tokenizer.is_fast

# check the same attribute of encoding
encoding.is_fast

True

Let’s see what a **fast tokenizer** enables us to do.

First, we can **access the tokens without having to convert the IDs back to tokens**:

In [3]:
encoding.tokens()
# ['[CLS]', 'My', 'name', 'is', 'S', '##yl', '##va', '##in', 'and', 'I', 'work', 'at', 'Hu', '##gging', 'Face', 'in', 'Brooklyn', '.', '[SEP]']

['[CLS]',
 'My',
 'name',
 'is',
 'S',
 '##yl',
 '##va',
 '##in',
 'and',
 'I',
 'work',
 'at',
 'Hu',
 '##gging',
 'Face',
 'in',
 'Brooklyn',
 '.',
 '[SEP]']

We can also use the `word_ids()` method to get the index of the word each token comes from

In [4]:
encoding.word_ids()

[None, 0, 1, 2, 3, 3, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, None]

We can see that the tokenizer’s special tokens **[CLS]** and **[SEP]** are mapped to None, and then each token is mapped to the word it originates from.

This is especially useful to determine if a token is at the start of a word or if two tokens are in the same word. We could rely on the `##` prefix for that, but it only works for BERT-like tokenizers; this method works for any type of tokenizer as long as it’s a fast one. We can also use it to mask all the tokens coming from the same word in masked language modeling (a technique called *whole word masking*).

Lastly, we can **map any word or token to characters in the original text**, and *vice versa*, via the `word_to_chars()` or `token_to_chars()` and `char_to_word()` or `char_to_token()` methods. For instance, the `word_ids()` method told us that ##yl is part of the word at index 3, but which word is it in the sentence?

In [5]:
start, end = encoding.word_to_chars(3)
example[start:end]

'Sylvain'

As we mentioned previously, this is all **powered by the fact the *fast tokenizer* keeps track of the span of text each token comes from in a list of offsets**.

In [22]:
## Example
example2 = "My name is Aditya, currently exploring NLP and Transformers in HuggingFace"
encoding2 = tokenizer(example2)
print(type(encoding2))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [23]:
print(encoding2.tokens())
print(encoding2.word_ids())

['[CLS]', 'My', 'name', 'is', 'Ad', '##ity', '##a', ',', 'currently', 'exploring', 'NL', '##P', 'and', 'Transformers', 'in', 'Hu', '##gging', '##F', '##ace', '[SEP]']
[None, 0, 1, 2, 3, 3, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 11, None]


In [24]:
start, end = encoding2.word_to_chars(11)
example2[start:end]

'HuggingFace'

### **Inside the token-classification pipeline**
- Chapter 1: got our first taste of applying NER — where the task is to identify which parts of the text correspond to entities like persons, locations, or organizations — with the 🤗 Transformers `pipeline()` function.
- Chapter 2, we saw how a `pipeline` groups together the three stages necessary to get the predictions from a raw text: *tokenization, passing the inputs through the model, and post-processing*.

The first two steps in the `token-classification` pipeline are the same as in any other pipeline, but the post-processing is a little more complex.


#### **Getting the base results with the pipeline**
Grab a *token classification* pipeline so we can get some results to compare manually. The model used by default is `dbmdz/bert-large-cased-finetuned-conll03-english`; it performs `NER` on sentences.

In [25]:
from transformers import pipeline

token_classifier = pipeline("token-classification")
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

[{'entity': 'I-PER',
  'score': 0.99938285,
  'index': 4,
  'word': 'S',
  'start': 11,
  'end': 12},
 {'entity': 'I-PER',
  'score': 0.99815494,
  'index': 5,
  'word': '##yl',
  'start': 12,
  'end': 14},
 {'entity': 'I-PER',
  'score': 0.99590707,
  'index': 6,
  'word': '##va',
  'start': 14,
  'end': 16},
 {'entity': 'I-PER',
  'score': 0.99923277,
  'index': 7,
  'word': '##in',
  'start': 16,
  'end': 18},
 {'entity': 'I-ORG',
  'score': 0.9738931,
  'index': 12,
  'word': 'Hu',
  'start': 33,
  'end': 35},
 {'entity': 'I-ORG',
  'score': 0.976115,
  'index': 13,
  'word': '##gging',
  'start': 35,
  'end': 40},
 {'entity': 'I-ORG',
  'score': 0.9887976,
  'index': 14,
  'word': 'Face',
  'start': 41,
  'end': 45},
 {'entity': 'I-LOC',
  'score': 0.9932106,
  'index': 16,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

The model properly identified each token generated by “Sylvain” as a person, each token generated by “Hugging Face” as an organization, and the token “Brooklyn” as a location.

We can also ask the pipeline to group together the tokens that correspond to the same entity:

In [28]:
token_classifier = pipeline("token-classification", aggregation_strategy="simple")
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity_group': 'PER',
  'score': 0.9981694,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9796019,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9932106,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

The `aggregation_strategy` picked will change the scores computed for each grouped entity. With `"simple"` the score is just the *mean of the scores of each token in the given entity*: for instance, the score of “Sylvain” is the mean of the scores we saw in the previous example for the tokens `S`, `##yl`, `##va`, and `##in`.

The other strategies are: `"first"`, `"max"`, `"average"`

Now let’s see how to obtain these results without using the pipeline() function!

#### **From Inputs to Prediction**
*First* we need to **tokenize our input and pass it through the model**. We instantiate the tokenizer and the model using the `AutoXxx` classes and then use them on our example.

In [29]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
inputs = tokenizer(example, return_tensors="pt")
outputs = model(**inputs)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Since we’re using `AutoModelForTokenClassification` here, we get one set of `logits` for each token in the input sequence

In [31]:
inputs

{'input_ids': tensor([[  101,  1422,  1271,  1110,   156,  7777,  2497,  1394,  1105,   146,
          1250,  1120, 20164, 10932, 10289,  1107,  6010,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [30]:
print(inputs["input_ids"].shape)
print(outputs.logits.shape)

torch.Size([1, 19])
torch.Size([1, 19, 9])


We have a batch with *1 sequence* of *19 tokens* and the model has *9 different labels*, so the output of the model has a shape of `1 x 19 x 9`. Like for the text classification pipeline, we use a `softmax` function to *convert those logits to probabilities*, and we take the `argmax` *to get predictions* (note that we can take the argmax on the logits because the softmax does not change the order)

In [32]:
import torch

probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0].tolist() # logits to probabilities
predictions = outputs.logits.argmax(dim=-1)[0].tolist() # get the prediction
print(probabilities)
print(predictions)

[[0.9994322657585144, 1.6470330592710525e-05, 3.4267097362317145e-05, 1.6042358765844256e-05, 8.25070746941492e-05, 2.1382355043897405e-05, 0.00015649135457351804, 1.965215415111743e-05, 0.00022089284902904183], [0.9989631175994873, 1.8515771444072016e-05, 5.240457539912313e-05, 1.253474511031527e-05, 0.0004347368376329541, 3.0874354706611484e-05, 0.0003146878443658352, 2.786072582239285e-05, 0.00014510878827422857], [0.999708354473114, 8.308103133458644e-06, 2.8745558665832505e-05, 5.650347247865284e-06, 8.694831922184676e-05, 9.783449058886617e-06, 6.786132144043222e-05, 1.1793958947237115e-05, 7.241879211505875e-05], [0.9998350143432617, 5.645520559482975e-06, 1.3955125723441597e-05, 4.3133691178809386e-06, 4.017683386337012e-05, 8.123054612951819e-06, 5.648485239362344e-05, 8.991617505671456e-06, 2.7239060727879405e-05], [0.00018333387561142445, 2.5156570700346492e-05, 4.846194133278914e-05, 1.4900567293807399e-05, 0.9993828535079956, 1.9997702111140825e-05, 0.00011153610103065148,

The `model.config.id2label` attribute contains the **mapping of indexes to labels** that we can use to make sense of the predictions

In [33]:
model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}

As we saw earlier, there are 9 labels: `O` is the label for the tokens that are not in any named entity (it stands for “outside”), and we then have two labels for each type of entity (miscellaneous, person, organization, and location). The label `B-XXX` indicates the token is at the beginning of an entity `XXX` and the label `I-XXX` indicates the token is inside the entity `XXX`.

📓 For instance, in the current example we would expect our model to classify the token `S` as `B-PER` (beginning of a person entity) and the tokens `##yl`, `##va` and `##in` as `I-PER` (inside a person entity).

With this map, we are ready to reproduce (almost entirely) the results of the first `pipeline` — we can just grab the score and label of each token that was not classified as `O`:

In [41]:
results = []
tokens = inputs.tokens()

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        results.append(
            {"entity": label, "score": probabilities[idx][pred], "word": tokens[idx]}
        )

print(results)

[{'entity': 'I-PER', 'score': 0.9993828535079956, 'word': 'S'}, {'entity': 'I-PER', 'score': 0.9981548190116882, 'word': '##yl'}, {'entity': 'I-PER', 'score': 0.995907187461853, 'word': '##va'}, {'entity': 'I-PER', 'score': 0.9992327690124512, 'word': '##in'}, {'entity': 'I-ORG', 'score': 0.9738931059837341, 'word': 'Hu'}, {'entity': 'I-ORG', 'score': 0.9761149883270264, 'word': '##gging'}, {'entity': 'I-ORG', 'score': 0.9887974858283997, 'word': 'Face'}, {'entity': 'I-LOC', 'score': 0.99321049451828, 'word': 'Brooklyn'}]


This is very similar to what we had before, with one exception: the `pipeline` also gave us information about the `start` and `end` of each entity in the original sentence. This is where our *offset mapping* will come into play. To get the offsets, we just have to set `return_offsets_mapping=True` when we apply the tokenizer to our inputs

In [42]:
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
inputs_with_offsets["offset_mapping"]

[(0, 0),
 (0, 2),
 (3, 7),
 (8, 10),
 (11, 12),
 (12, 14),
 (14, 16),
 (16, 18),
 (19, 22),
 (23, 24),
 (25, 29),
 (30, 32),
 (33, 35),
 (35, 40),
 (41, 45),
 (46, 48),
 (49, 57),
 (57, 58),
 (0, 0)]

"My name is Sylvain and I work at Hugging Face in Brooklyn."

Each tuple is the span of text corresponding to each token, where `(0, 0)` is reserved for the special tokens. We saw before that the token at index 5 is `##yl`, which has `(12, 14)` as offsets here.

If we grab the corresponding slice in our example

In [47]:
example[12:14]

'yl'

Using this we can complete the previous results

In [48]:
results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        start, end = offsets[idx]
        results.append(
            {
                "entity": label,
                "score": probabilities[idx][pred],
                "word": tokens[idx],
                "start": start,
                "end": end,
            }
        )

print(results)

[{'entity': 'I-PER', 'score': 0.9993828535079956, 'word': 'S', 'start': 11, 'end': 12}, {'entity': 'I-PER', 'score': 0.9981548190116882, 'word': '##yl', 'start': 12, 'end': 14}, {'entity': 'I-PER', 'score': 0.995907187461853, 'word': '##va', 'start': 14, 'end': 16}, {'entity': 'I-PER', 'score': 0.9992327690124512, 'word': '##in', 'start': 16, 'end': 18}, {'entity': 'I-ORG', 'score': 0.9738931059837341, 'word': 'Hu', 'start': 33, 'end': 35}, {'entity': 'I-ORG', 'score': 0.9761149883270264, 'word': '##gging', 'start': 35, 'end': 40}, {'entity': 'I-ORG', 'score': 0.9887974858283997, 'word': 'Face', 'start': 41, 'end': 45}, {'entity': 'I-LOC', 'score': 0.99321049451828, 'word': 'Brooklyn', 'start': 49, 'end': 57}]


This is the same as what we got from the first pipeline!

#### **Grouping Entities**
Using the `offsets` to determine the start and end keys for each entity is handy, but that information isn’t strictly necessary.

When we want to group the entities together, however, the offsets will save us a lot of messy code. For example, if we wanted to group together the tokens `Hu`, `##gging`, and `Face`, we could make special rules that say *the first two should be attached while removing the `##`*, and *the Face should be added with a space since it does not begin with `##`* — but that would only work for this particular type of tokenizer ➡➡➡ [*another set rules for a `SentecePiece` or a `Byte-Pair-Encoding Tokenizer`*]

With the offsets, all that custom code goes away: we just can **take the span in the original text that begins with the first token and ends with the last token**. So, in the case of the tokens `Hu`, `##gging`, and `Face`, we should start at character 33 (the beginning of `Hu`) and end before character 45 (the end of `Face`):

In [49]:
example[33:45]

'Hugging Face'

To write the code that *post-processes the predictions while grouping entities*, we will **group together entities that are consecutive and labeled with `I-XXX`, except for the first one, which can be labeled as `B-XXX` or `I-XXX`** (so, we stop grouping an entity when we get a O, a new type of entity, or a `B-XXX` that tells us an entity of the same type is starting):

In [53]:
import numpy as np

results = [] # save the results
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True) # tokenizer with offset
tokens = inputs_with_offsets.tokens() # get the tokens of input
offsets = inputs_with_offsets["offset_mapping"] # slice for the offset value only

idx = 0
while idx < len(predictions):
    pred = predictions[idx] # get the prediction value by index
    label = model.config.id2label[pred] # get the label of the model from prediction values
    if label != "O":
        # Remove the B- or I-
        label = label[2:]
        start, _ = offsets[idx]

        # Grab all the tokens labeled with I-label
        all_scores = []
        while (
            idx < len(predictions)
            and model.config.id2label[predictions[idx]] == f"I-{label}"
        ):
            all_scores.append(probabilities[idx][pred])
            _, end = offsets[idx]
            idx += 1

        # The score is the mean of all the scores of the tokens in that grouped entity
        score = np.mean(all_scores).item()
        word = example[start:end]
        results.append(
            {
                "entity_group": label,
                "score": score,
                "word": word,
                "start": start,
                "end": end,
            }
        )
    idx += 1

print(results)

[{'entity_group': 'PER', 'score': 0.998169407248497, 'word': 'Sylvain', 'start': 11, 'end': 18}, {'entity_group': 'ORG', 'score': 0.9796018600463867, 'word': 'Hugging Face', 'start': 33, 'end': 45}, {'entity_group': 'LOC', 'score': 0.99321049451828, 'word': 'Brooklyn', 'start': 49, 'end': 57}]


And we get the same results as with our second pipeline!

Another example of a task where these `offsets` are extremely useful is *question answering*.

## **Fast Tokenizers in the `QA Pipeline`**

## **`Normalization` and Pre-Tokenization**

## **`Byte-Pair Encoding` Tokenization**

## **`WordPiece` Tokenization**

## **`Unigram` Tokenization**

## **Building a Tokenizer, block-by-block**